## Import libraries

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gurobipy import *

## Read and preprocess data


In [26]:
import csv
from datetime import datetime
import os

def load_data(file_path):
    data_dict = dict()

    with open(file_path) as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['Numéro du groupe'] != '':
                heures, minutes, _ = map(int, row['TransitTime'].split(':')) 
                total_minutes = heures * 60 + minutes
                data_dict[int(float(row['Numéro du groupe']))] = (
                    int(float(row['Femmes'])),
                    int(float(row['Hommes'])),
                    int(float(row['WCHR'])),
                    total_minutes
                )

    return data_dict

# Load data for each file

file_names = ["5Nov.csv", "7Nov.csv", "22Oct.csv", "24Oct.csv", "23Oct.csv", "30Oct.csv", "DataSeating 2024.csv"]
data_directory = 'data'
# for eachy dictionary : dict[Numéro du groupe : (Femmes, Hommes, WCHR, TransitTime)]
Nov5, Nov7, Oct22, Oct24, Oct23, Oct30, Date0 = [load_data(os.path.join(data_directory, file)) for file in file_names]

Example of how we access data

In [27]:
Nov5[1][0] # Number of women in group 1 on November 5th

1

Transforming data to access each passenger's Id

First we solved the problem for November 5th, then we'll have to automate it

In [28]:
Passengers = {}
j = 1

for group_num, data in Nov5.items():
    weights = {'Femmes': 70, 'Hommes': 85, 'WCHR': 100}  # Weight per passenger
    
    # Extracting data
    femmes, hommes, wchr, total_minutes = data
    
    # Enumerating passengers
    for _ in range(femmes + hommes + wchr):
        if femmes > 0:
            weight = weights['Femmes']  
            Passengers[j] = {'gender': 'Femmes', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            femmes -= 1
        elif hommes > 0:
            weight = weights['Hommes']
            Passengers[j] = {'gender': 'Hommes', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            hommes -= 1
        else:
            weight = weights['WCHR']
            Passengers[j] = {'gender': 'WCHR', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            wchr -= 1
        j += 1

print(Passengers)

{1: {'gender': 'Femmes', 'group': 1, 'weight': 70, 'connection_time': 0}, 2: {'gender': 'Hommes', 'group': 1, 'weight': 85, 'connection_time': 0}, 3: {'gender': 'Hommes', 'group': 1, 'weight': 85, 'connection_time': 0}, 4: {'gender': 'Hommes', 'group': 2, 'weight': 85, 'connection_time': 60}, 5: {'gender': 'Femmes', 'group': 3, 'weight': 70, 'connection_time': 165}, 6: {'gender': 'Hommes', 'group': 3, 'weight': 85, 'connection_time': 165}, 7: {'gender': 'Femmes', 'group': 4, 'weight': 70, 'connection_time': 0}, 8: {'gender': 'Hommes', 'group': 4, 'weight': 85, 'connection_time': 0}, 9: {'gender': 'WCHR', 'group': 4, 'weight': 100, 'connection_time': 0}, 10: {'gender': 'Femmes', 'group': 5, 'weight': 70, 'connection_time': 0}, 11: {'gender': 'Femmes', 'group': 5, 'weight': 70, 'connection_time': 0}, 12: {'gender': 'Femmes', 'group': 5, 'weight': 70, 'connection_time': 0}, 13: {'gender': 'Femmes', 'group': 6, 'weight': 70, 'connection_time': 0}, 14: {'gender': 'Hommes', 'group': 6, 'weig

In [29]:
Passengers[1] # Characteristic of passenger 1 of November 5

{'gender': 'Femmes', 'group': 1, 'weight': 70, 'connection_time': 0}

## Static model

In [30]:
ranks = 29 #number of rows in the aircraft
n = 90 #number of passengers - TO BE CHANGED
ns=7*ranks #number of seats

def modele_statique():
    m=Model('statique') # Model initialization
    
    # -- Adding variables  --
    # Sij : dict[(int, int) : Var] : Associate passenger i with seat j
    S = {(i,j) : m.addVar(vtype = GRB.BINARY, name = f'j{i}') for i in range(1, n+1) for j in range (1, ns+1)}

    # -- Adding constraints  --
    
    # Max. 1 seat per passenger
    for j in range (1, ns+1):
        m.addConstr(sum(S[(i,j)] for i in range (1, n+1)) <=1, name="PassengerMax")

    #Each passenger has one and only one seat
    for i in range (1, n+1):
        m.addConstr(sum(S[(i,j)] for j in range (1, ns+1)) == 1, name="SeatMax")

    #The central aisle is left free
    for i in range (1, n+1):
        m.addConstr(S[(i, 4)] == 0)

    #Barycenter
    x_g = LinExpr()
    y_g = LinExpr()

    total_weight = quicksum(Passengers[i]['weight'] for i in range(1, n + 1)).getValue()

    for i in range(1, n + 1):
        for j in range(1, ns + 1):
            weight_ij = Passengers[i]['weight'] / total_weight
            if (j % 7 == 0):
                x_g += (1 * S[i, j] * weight_ij)
                y_g += (1 * S[i, j] * weight_ij)
            else:
                x_g += ((j % 7) * S[i, j] * weight_ij)
                y_g += ((j % 21) * S[i, j] * weight_ij)
    
    

    m.addConstr(x_g >= 3)  
    m.addConstr(x_g <= 5)  
    m.addConstr(y_g >= 13)  
    m.addConstr(y_g <= 17)

    return m, S

Fonctions objectifs

In [31]:
## Fonctions objectifs

m,S=modele_statique()

#Les passagers en transit sont placés à l'avant de l'avion

def obj_transit():
    T= {} 
    
    
    for passenger in Passengers:
        if Passengers[passenger]['connection_time'] >0:   #passager en transit
            T[passenger]=Passengers[passenger]['connection_time']

    #T=sorted(T.items(), key=lambda item:item[1], reverse=True)
    P=list(T.keys())  #retourne la liste des identifiants des passagers en transit
    

    f=0
    for k in range (len(P)):
        for j in range (1, ns+1):
            q=(j-1)//7
            f+= S[(P[k],j)] * (1/T[P[k]]) * q
            

    return f

m.setObjective(obj_transit(), GRB.MINIMIZE)

In [32]:
# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet

# -- Mise à jour du modèle  --
m.update() 

# -- Affichage en mode texte du PL --
display(m)

<gurobi.Model MIP instance statique: 387 constrs, 18270 vars, Parameter changes: Username=(user-defined), OutputFlag=0>

In [33]:
# -- Résolution --
m.optimize()

# -- Affichage de la solution --
print("Les places dans l'avion sont les suivantes :", [(i,j) for i in range(1,n+1) for j in range (1,ns+1) if S[(i,j)].x])

Les places dans l'avion sont les suivantes : [(1, 103), (2, 132), (3, 95), (4, 1), (5, 25), (6, 15), (7, 147), (8, 62), (9, 143), (10, 163), (11, 78), (12, 134), (13, 178), (14, 166), (15, 16), (16, 10), (17, 187), (18, 11), (19, 2), (20, 6), (21, 3), (22, 104), (23, 199), (24, 162), (25, 57), (26, 158), (27, 139), (28, 125), (29, 48), (30, 74), (31, 184), (32, 8), (33, 180), (34, 91), (35, 146), (36, 124), (37, 138), (38, 188), (39, 102), (40, 60), (41, 120), (42, 76), (43, 101), (44, 174), (45, 61), (46, 53), (47, 190), (48, 99), (49, 181), (50, 20), (51, 17), (52, 9), (53, 198), (54, 186), (55, 159), (56, 114), (57, 38), (58, 34), (59, 32), (60, 41), (61, 141), (62, 122), (63, 164), (64, 55), (65, 201), (66, 18), (67, 13), (68, 165), (69, 52), (70, 7), (71, 5), (72, 121), (73, 83), (74, 39), (75, 36), (76, 183), (77, 117), (78, 81), (79, 19), (80, 167), (81, 185), (82, 189), (83, 118), (84, 37), (85, 14), (86, 12), (87, 21), (88, 40), (89, 80), (90, 200)]
